In [1]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers.experimental.preprocessing import CenterCrop

from tensorflow.keras import layers
import numpy as np
import tensorflow as tf
from tensorflow import keras


from dataset_manager import *
from predict import *

train_dataset, test_dataset = get_dataset()
d_list = list(test_dataset)

X_train, Y_train = get_X_Y(train_dataset)

X_test, Y_test = get_X_Y(test_dataset)

#X = []
#for i in range(0, len(Y_train) - 1):
#    X.append([Y_train[i], X_train[i]])




In [ ]:
from sklearn.datasets import make_regression
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.models import Sequential
from keras.optimizers import SGD
from matplotlib import pyplot
from dataset_manager import *
from predict import *

train_dataset, test_dataset = get_dataset()
X_train, Y_train = get_X_Y_vectorized_int(train_dataset)
X_test, Y_test = get_X_Y_vectorized_int(test_dataset)

X_train, Y_train = shuffle_X_Y(X_train, Y_train)
X_test, Y_test = shuffle_X_Y(X_test, Y_test)

model = Sequential()
#model.add(Flatten(input_shape=(len(X_train), 17)))
model.add(Dense(17))
for _ in range(6):
    model.add(Dense(128, activation='sigmoid'))
model.add(Dense(42))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])


model.fit(np.array(X_train), np.array(Y_train), epochs=10)
model.summary()

Epoch 1/10
7875/7875 [==============================] - 8s 1ms/step - loss: 0.0237 - accuracy: 0.1227
Epoch 2/10
7875/7875 [==============================] - 8s 1ms/step - loss: 0.0148 - accuracy: 0.4478
Epoch 3/10
7875/7875 [==============================] - 8s 1ms/step - loss: 0.0137 - accuracy: 0.4831
Epoch 4/10
7875/7875 [==============================] - 8s 1ms/step - loss: 0.0132 - accuracy: 0.4993
Epoch 5/10
7875/7875 [==============================] - 8s 1ms/step - loss: 0.0130 - accuracy: 0.5087
Epoch 6/10
4594/7875 [================>.............] - ETA: 3s - loss: 0.0128 - accuracy: 0.5178

In [27]:
test_loss, test_acc = model.evaluate(np.array(X_test), np.array(Y_test), verbose=3)

print('\nTest accuracy:', test_acc)

ValueError: in user code:

    /usr/local/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1233 test_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1224 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1217 run_step  **
        outputs = model

In [ ]:
def get_prediction_list(model, pic="tohack/pics_LOGINMDP"):
    """
    The model is like "model = LinearRegression()" (or other algorithms)
    """

    x_pred, info = get_pics_from_file(f"../{pic}.bin")
    
    res = []

    preds_brut = model.predict(np.array(x_pred))
    for pred in preds_brut:
        itemindex = np.where(pred==max(pred))[0][0]
        prediction = d_list[itemindex]
        if "data" in pic or prediction != 'NOKEY':
            res.append(prediction)
    return res


def get_creds(prediction_list):
    for i_k in range(len(prediction_list)):
        if prediction_list[i_k] == 'CTRL' and i_k + 2 < len(prediction_list) and prediction_list[i_k + 2] == 'SUPPR':
            print('#####################')
            print(prediction_list[i_k:i_k + 42])


In [ ]:
def compute_accuracy():
    accuracy = {}
    accuracy_count = {}
    
    preds_brut = model.predict(np.array(X_test))
    
    for pred, y in zip(preds_brut, Y_test):
        realitemindex = np.where(y==1)[0][0]
        key = d_list[realitemindex]
        itemindex = np.where(pred==max(pred))[0][0]
        
        if realitemindex == itemindex:
            if key not in accuracy:
                accuracy[key] = 0
                accuracy_count[key] = 0
            accuracy[key] += 1
        if key not in accuracy_count:
            accuracy_count[key] = 0
        accuracy_count[key] += 1


    for key in accuracy:
        accuracy[key] /= accuracy_count[key]

    return accuracy
    
accuracy = compute_accuracy()
for key, acc in accuracy.items():
    print(key, acc)
    
print("accuracy: ", sum(accuracy.values()) / 42)

In [ ]:
res = get_prediction_list(model)
print(res)

In [ ]:
get_creds(res)